<a href="https://colab.research.google.com/github/shankartmv/Deep-Learning-Work/blob/main/IMDB_Sentiment_reviews_using_tensorflow_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import seaborn as sns
import chardet
import keras.utils.vis_utils as kutils
from operator import itemgetter
from keras.initializers import Constant

In [ ]:
print(tf.__version__)
print(tf.executing_eagerly())
print('Size of training dataset = ',train_dataset.cardinality(),'\nSize of test dataset = ',test_dataset.cardinality())

In [ ]:
train_dataset,train_info=tfds.load('imdb_reviews',with_info=True,split='train+test[:80%]')
test_dataset,test_info=tfds.load('imdb_reviews',with_info=True,split='test[80%:100%]')

In [ ]:
print('Train dataset type = ',type(train_dataset),'\nTest dataset type = ',type(test_dataset))
# Elements under the dataset is again a dictionary of tensor objects(values)
print('Train dataset contents = ',train_dataset.element_spec,'\nTrain dataset contents = ',test_dataset.element_spec)